In [3]:
import numpy as np
import matplotlib.pyplot as plt
def loadDataSet(fileName):
    numFeat = len(open(fileName).readline().split('\t'))-1
    xArr = []
    yArr = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = []
        currLine = line.strip().split('\t')
        for i in range(numFeat):
            lineArr.append(float(currLine[i]))
        xArr.append(lineArr)
        yArr.append(float(currLine[-1]))
    return xArr,yArr
def standRegres(xArr,yArr):
    xMat = mat(xArr)
    yMat = mat(yArr)
    xTx = xMat.T * xMat
    if linalg.det(xTx)==0.0:
        print('this matrix is singular, cannot do inverse')
        return 
    ws = xTx.I * (xMat.T*yMat)
    return ws
def plotFigure(xArr,yArr,yHat):
    xMat = mat(xArr)
    yMat = mat(yArr)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(xMat[:,1].flatten().A[0],yMat.T[:,0].flatten().A[0],s=2,c='red')
    srtInd = xMat[:,1].argsort(0)
    xSort = xMat[srtInd][:,0,:]
    ax.plot(xSort[:,1],yHat[srtInd])
    plt.show()

In [4]:
def lwlr(testPoints,xArr,yArr,k=1.0):
    xMat = mat(xArr)
    yMat = mat(yArr)
    m = shape(xMat)[0]
    weights = mat(eye((m)))
    for j in range(m):
        diffMat = testPoints-xMat[j,:]
        weights[j,j]=exp(diffMat*diffMat.T(-2*k**2))
    xTx = xMat.T*(weights*xMat)
    if linalg.det(xTx)==0.0:
        print('this matrix is sigular cannot do inverse')
        return 
    ws = xTx.I*(xMat.T*(weights*yMat))
    return testPoints * ws
def lwlrTest(testArr,xArr,yArr,k=1.0):
    m = shape(testArr)[0]
    yhat = zeros(m)
    for i in range(m):
        yHat[i] = lwlr(testArr[i],xArr,yArr,k)
    return yHat
        

In [6]:
def rssError(yArr,yHatArr):
    return ((yArr-yHatArr)**2).sum()

In [9]:
def ridgeRegres(xMat,yMat,lam=0.2):
    xTx = xMat.T*xMat
    denom = xTx +eye(shape(xMat)[1])*lam
    if linalg.det(denom)==0.0:
        print('this matrix is singular,cannot do inverse')
        return 
    ws = denom.I * (xMat.T*yMat)
    return ws

def regularize(xMat):
    retMat = xMat.copy()
    xMeans = mean(retMat,0)
    xVar = var(retMat,0)
    retMat = (retMat-xMeans)/xVar
    return retMat

In [11]:
def ridgeTest(xArr,yArr):
    xMat = mat(xArr)
    yMat = mat(yArr).T
    yMaen = mean(yMat,0)
    yMat = yMat - yMean
    xMat = regularize(xMat)
    numTestPts = 30
    wMat = zeros((numTestPts,shape(xMat)[1]))
    for i in range(numTestPts):
        ws = ridgeRegres(xMat,yMat,exp(i-10))
        wMat[i,:]=ws.T
    return wMat

In [12]:
def stageWise(xArr,yArr,eps=0.01,numIt=100):
    xMat = mat(xArr)
    yMat = mat(yArr).T
    yMean = mean(yMat,0)
    yMat = yMat - yMean
    xMat = regularize(xMat)
    m,n = shape(xMat)
    returnMat = zeros((numIt,n))
    ws = zeros((n,1))
    wsBest = ws.copy()
    for i in range(numIt):
        print(ws.T)
        lowestError = inf
        for j in range(n):
            for sign in [-1,1]:
                wsTest = ws.copy()
                wsTest[j]+= eps*sign
                yTest = xMat * wsTest
                rssE = rssError(yMat.A,yTest.A)
                if rssE<lowestError:
                    lowestError = rssE
                    wsBest = wsTest
        ws = wsBest.copy()
        returnMat[i,:]=ws.T
    return returnMat